## Cal Adapt flooding exploration
_Goals:_

(1) Model which crops are the most at risk of flooding?

(2) Understand: Can we identify (crop) land that might be most at risk of climate change-induced flooding?

In [4]:
# Get the data from the API
import requests
import pandas as pd

In [5]:
# Get lists of datasets in Cal-Adapt API

response = requests.get('http://api.cal-adapt.org/api/datasets/?pagesize=20')
response = response.json()
datasets = pd.DataFrame(response['results'])
datasets

,url,gcm,scenarios,publisher,resources,tags,title,description
0,https://api.cal-adapt.org/api/datasets/b3b4f7a...,[],[],University of California Berkeley,[{'name': 'Cal-Adapt Data Server - calflod-3d'...,"[coastal flooding, sea level rise]",CalFloD-3D,Inundation extent and depth layers for the San...
1,https://api.cal-adapt.org/api/datasets/79d4535...,[],[],Geospatial Innovation Facility,[{'name': 'Cal-Adapt Data Server - calflod-3d-...,"[coastal flooding, sea level rise]",CalFloD-3D Mosaics,Mosaics of the original CalFloD-3D inundation ...
2,https://api.cal-adapt.org/api/datasets/a79fafa...,"[CNRM-CM5, CanESM2, HadGEM2-ES, MIROC5]","[RCP 4.5, RCP 8.5]",University of California Berkeley,[{'name': 'Cal-Adapt Data Server - calflod3d-t...,"[coastal flooding, sea level rise]",CalFloD3D-TFS,Inundation layers representing coastal floodin...
3,https://api.cal-adapt.org/api/datasets/5f5c10e...,[],[],Geospatial Innovation Facility,[{'name': 'Cal-Adapt Data Server - calflod3d-t...,"[coastal flooding, sea level rise]",CalFloD3D-TFS Mosaics,Mosaics of selected combinations of the 50 m a...
4,https://api.cal-adapt.org/api/datasets/23ecb48...,[],[],United States Geological Survey,"[{'name': 'Cal-Adapt Data Server - cosmos', 'u...","[coastal flooding, sea level rise]",CoSMoS,Coastal flooding layers for 100 year storm sce...
5,https://api.cal-adapt.org/api/datasets/ccb284d...,[],[],Geospatial Innovation Facility,[{'name': 'Cal-Adapt Data Server - cosmos-mosa...,"[coastal flooding, sea level rise]",CoSMoS Mosaics,Mosaics of selected combinations of the CoSMoS...
6,https://api.cal-adapt.org/api/datasets/8810b3f...,"[CNRM-CM5, GFDL-CM3, INMCM4, IPSL-CM5A-LR, MPI...",[RCP 8.5],UCLA Center for Climate Science,"[{'name': 'Cal-Adapt Data Server - ucla', 'url...","[air temperature, precipitation, snow water eq...",Dynamical Downscaling Product,The UCLA Center for Climate Science used the W...
7,https://api.cal-adapt.org/api/datasets/0887cba...,[],[Historical],University of Colorado Boulder,"[{'name': 'Cal-Adapt Data Server - livneh', 'u...","[maximum temperature, minimum temperature, pre...",Gridded Observed Meteorological Data,Historical observed daily temperature data fro...
8,https://api.cal-adapt.org/api/datasets/18e1cb9...,[],[Historical],Geospatial Innovation Facility,"[{'name': 'Thomas, N., Mukhtyar, S., Galey, B....","[KBDI, dry spell, extreme heat, maximum temper...",Gridded Observed Meteorological Data Derived P...,These datasets were generated from the Gridded...
9,https://api.cal-adapt.org/api/datasets/b0ad452...,[],[Historical],University of Colorado Boulder,[{'name': 'Cal-Adapt Data Server - livneh_vic-...,"[air temperature, albedo, baseflow, dew, evapo...",Gridded Observed Meteorological Data VIC Runs,The Gridded Observed meteorological data (dail...


In [6]:
with open('datasets-urls-caladapt.txt', 'w') as f:
    f.write(datasets['url'].to_string())

_Goal_: Get rainfall/precipitation data. Ideal sampling specifications: space-wise: localized (1km radius); time-wise: shortest timespan possible (daily? > monthly? > yearly?)

_Motivation_: "The potential impacts of heavy precipitation include crop damage, soil erosion, and an increase in flood risk due to heavy rains" (EPA, n.d.) https://www.epa.gov/climate-indicators/climate-change-indicators-heavy-precipitation

In [7]:
datasets.columns

Index(['url', 'gcm', 'scenarios', 'publisher', 'resources', 'tags', 'title',
       'description'],
      dtype='object')

In [8]:
keywords = ['rainfall', 'precipitation']
any(keyword in datasets['tags'][15] for keyword in keywords)

True

In [78]:
# Identify the rows (i.e. datasets) that include either "precipitation" or "rainfall" in the 'tags' column
dfs = pd.DataFrame()
dfs = pd.concat([datasets.loc[[i]] for i in range(len(datasets)) if any(tag in datasets.loc[i]['tags'] for tag in ['precipitation', 'rainfall'])]).reset_index(drop=True)

# j = 0
# for i in range(len(datasets)):
#     if any(tag in datasets.loc[i]['tags'] for tag in ['precipitation', 'rainfall']):
#         dfs[j] = datasets.loc[i]
#         j += 1

In [86]:
# # Print the description without shortening the line
# print(f'{len(dfs.loc["description"])=}')
# for i in range(len(dfs.loc['description'])):
#     print('\n', i, dfs.loc['description'][i])

In [11]:
class PrecipDataset:
    """Cal-Adapt-documented dataset on precipitation and its metadata"""
    def __init__(self, index: int, name: str, description: str, url: str, temp_freq: str = None, timespan: list() = None, spat_res: str = None):
        """Create a new dataset documentation with metadata (parameters/arguments)

        Args:
            df_index (int): index in the Cal-Adapt API documentation
            name (str): name of the dataset in Cal-Adapt API documentation
            description (str): description of the dataset in Cal-Adapt API documentation
            temp_freq (str): daily, monthly, yearly, yearly+
            timespan (list): dates of coverage of the dataset
            spat_res (str): sampling resolution of the dataset
        """
        
        self.df_index = index
        self.name = name
        self.description = description
        self.url = url
        self.temp_freq = temp_freq
        self.timespan = timespan
        self.spat_res = spat_res

In [12]:
# Instantiate most relevant datasets: 1, 2, 3, 4, 5, 7
## Instantiate #5 first

# dfs.loc[0] = [1, dfs.loc['title'][1], dfs.loc['description'][1], 'daily?', '1950-2013', 6]
# dfs.loc[1] = [2, dfs.loc['title'][2], dfs.loc['description'][2]]


In [ ]:
params = {
    'intersects': geojson,
    'srs': 4326,
    'precision': 4
}

In [108]:
print("Getting API data for: ", dfs.loc[5]['title'], 'at this url:', dfs.loc[5]['url'])
reponse = requests.get(dfs.loc[5]['url']) # params = params
response


Getting API data for:  Hourly Observed Historical Data at this url: https://api.cal-adapt.org/api/datasets/5e555315-da6a-4a49-a2a2-53408720ce08/


{'count': 25,
 'next': 'https://api.cal-adapt.org/api/datasets/?page=2&pagesize=20',
 'previous': None,
 'results': [{'url': 'https://api.cal-adapt.org/api/datasets/b3b4f7ae-182c-408b-8b7e-5365c44c6a92/',
   'gcm': [],
   'scenarios': [],
   'publisher': 'University of California Berkeley',
   'resources': [{'name': 'Cal-Adapt Data Server - calflod-3d',
     'url': 'http://albers.cnr.berkeley.edu/data/ucb/calflod-3d',
     'format': 'geotiff'},
    {'name': 'Radke et al. (2017). Assessment of Bay Area Natural Gas Pipeline Vulnerability to Climate Change. California Energy Commission. Publication number: CEC-500-2017-008.',
     'url': 'http://albers.cnr.berkeley.edu/data/ucb/calflod-3d/CEC-500-2017-008.pdf',
     'format': 'reference'},
    {'name': 'Stelling, G. S. (2012). Quadtree flood simulations with sub-grid digital elevation models.',
     'url': 'https://3diwatermanagement.com/wp-content/uploads/2018/11/Stelling_Quadtree-flood-simulations-with-sub-grid-digital-elevation-models.

In [ ]:
#    params = {
#         'intersects': geojson,
#         'srs': 4326,
#         'precision': 4
#     }

# response = requests.get(id_url, params=params)

#     if not response.ok:
#         return {'error': response.text}

#     data = response.json()
#     if len(data['features']) == 0:
#         return {'error': 'No features.'}

#     feature = data['features'][0]
#     if feature:
#         ref = '/api/%s/%s/' % (RESOURCE, feature['id'])
#         return {'error': None, 'ref': ref}
#     else:
#         return {'error': 'Intersection not found.'}

In [ ]:
# Desired output: data on flooding
# localized - 1km radius? 
# shortest term possible




In [ ]:
# Plot on map


In [ ]:
# Combine data with farmland data?

# Combine with elevation data?
